# Tagging genes with ddlite: candidate extraction

## Introduction
In this example **ddlite** app, we'll build a gene tagger from scratch. Here's why we developed ddlite:

* To provide a lighter-weight interface to structured information extraction for new DeepDive users
* To help advanced DeepDive rapidly develop and prototype applications and distant supervision rules
* To investigate DeepDive's data programming approach to building inference systems

This example is centered around the second item. Domain-specific tagging systems take months or years to develop. They use hand-crafted model circuitry and accurate, hand-labeled training data. We're going to try to build a pretty good one in a few minutes with none of those things. The generalized extraction and learning utilities provided by ddlite will allow us to turn a sampling of article abstracts and some basic domain knowledge into an automated tagging system. Specifically, we want an accurate tagger for genes in academic articles. We have comprehensive dictionaries of genes, but applying a simple matching rule might yield a lot of false positives. For example, "p53" might get tagged as a gene if it refers to a page number. Our goal is to use distant supervision to improve precision.

Here's the pipeline we'll follow:

1. Obtain and parse input data (relevant article abstracts from PubMed)
2. Extract candidates for tagging
3. Generate features
4. Write distant supervision rules
5. Learn the tagging model
6. Iterate on distant supervision rules

Parts 1 and 2 are covered in this notebook, and parts 3 through 6 are covered in `GeneTaggerExample_Learning.ipynb`. Let's get to it.

In [1]:
%load_ext autoreload
%autoreload 2

import cPickle
from ddlite import *

## Processing the input data
We already downloaded the raw HTML for 150 gene-related article pages from PubMed using the `pubmed_gene_html.py` script. These can be found in the `data` folder. We can use ddlite's `DocParser` to read in the article text. There's a general HTML parser which finds visible text, but we can do better by writing a more specific version to just grab the abstract text.

In [2]:
class PubMedAbstractParser(HTMLParser):
    def _cleaner(self, s):
        return (s.parent.name == 'abstracttext')

dp = DocParser('gene_tag_example/data/', PubMedAbstractParser())
docs = list(dp.parseDocs())
print docs[0]

Mutations in BCS1L, a respiratory chain complex III assembly chaperone, constitute a major cause of mitochondrial complex III deficiency and are associated with GRACILE and Bjrnstad syndromes. Here we describe a 4-year-old infant with hyperlactacidemia, mild liver dysfunction, hypotonia, growth and psychomotor retardation, dysmorphic features and mitochondrial complex III deficiency. Respiratory chain enzyme activities showed an isolated complex III defect in muscle and fibroblasts. Sequencing and polymerase chain reaction-restriction fragment length polymorphism (PCR-RFLP) analysis revealed a novel homozygous BCS1L mutation, c.148A>G, which caused a p.T50A substitution at an evolutionarily conserved BCS1L region. The severity of the complex III enzyme defect correlated with decreased amounts of BCS1L and respiratory chain complex III in the affected tissues. Our findings support a pathogenic role for the novel BCS1L mutation in a patient with a singular clinical phenotype.


Now we'll use CoreNLP via ddlite's `SentenceParser` to parse each sentence. `DocParser` can handle this too; we didn't really need that call above. This can take a little while, so if the example has already been run, we'll reload it.

In [3]:
docs = None

pkl_f = 'gene_tag_example/gene_tag_saved_sents_v2.pkl'
try:
    with open(pkl_f, 'rb') as f:
        sents = cPickle.load(f)
except:
    %time sents = dp.parseDocSentences()
    with open(pkl_f, 'w+') as f:
        cPickle.dump(sents, f)

print sents[0]

CPU times: user 12.6 s, sys: 248 ms, total: 12.8 s
Wall time: 24.2 s
Sentence(words=['Mutations', 'in', 'BCS1L', ',', 'a', 'respiratory', 'chain', 'complex', 'III', 'assembly', 'chaperone', ',', 'constitute', 'a', 'major', 'cause', 'of', 'mitochondrial', 'complex', 'III', 'deficiency', 'and', 'are', 'associated', 'with', 'GRACILE', 'and', 'Bjrnstad', 'syndromes', '.'], lemmas=['mutation', 'in', 'bcs1l', ',', 'a', 'respiratory', 'chain', 'complex', 'iii', 'assembly', 'chaperone', ',', 'constitute', 'a', 'major', 'cause', 'of', 'mitochondrial', 'complex', 'iii', 'deficiency', 'and', 'be', 'associate', 'with', 'gracile', 'and', 'bjrnstad', 'syndrome', '.'], poses=['NNS', 'IN', 'NN', ',', 'DT', 'JJ', 'NN', 'NN', 'CD', 'NN', 'NN', ',', 'VBP', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'NN', 'CD', 'NN', 'CC', 'VBP', 'VBN', 'IN', 'NN', 'CC', 'NN', 'NNS', '.'], dep_parents=[13, 3, 1, 3, 11, 11, 11, 11, 11, 11, 3, 3, 0, 16, 16, 13, 21, 21, 21, 21, 16, 13, 24, 13, 26, 24, 26, 29, 26, 13], dep_labels=['nsubj'

## Extracting candidates with matchers
Extracting candidates for mentions (or relations) in ddlite is done with `Matcher` objects. First, we'll use a `DictionaryMatcher`. We have access to a pretty comprehensive gene dictionary. Let's load it in and create the `DictionaryMatcher`.

In [4]:
# Schema is: ENSEMBL_ID | NAME | TYPE (refseq, canonical, non-canonical)
genes = [line.rstrip().split('\t')[1] for line in open('gene_tag_example/dicts/ensembl_genes.tsv')]
genes = filter(lambda g : len(g) > 2, genes)

dm = DictionaryMatch('GeneName', genes, ignore_case=False)

The dictionary match should provide fairly high recall, but we may still miss some candidates. We know that gene names are named nouns and are often all uppercase. Let's write our own matcher to catch all-uppercase named nouns.

In [5]:
class AllUpperNounsMatcher(Matcher):
    def __init__(self, label):
        self.label = label
        # Regex matcher to find named nouns in part-of-speech tags
        self._re_comp = re.compile("[A-Z]?NN[A-Z]?", flags=re.I)
    def apply(self, s):
        # Get parts-of-speech and words
        words = s.__dict__['words']
        pos = s.__dict__['poses']
        # Get all-cap words
        caps = set(idx for idx, w in enumerate(words) if w.upper() == w)
        # Convert character index to token index
        start_c_idx = [0]
        for s in pos:
            start_c_idx.append(start_c_idx[-1]+len(s)+1)
        # Find regex matches over phrase
        phrase = ' '.join(pos)
        for match in self._re_comp.finditer(phrase):
            # Get start and end indexes for tokens
            start = bisect.bisect(start_c_idx, match.start())
            end = bisect.bisect(start_c_idx, match.end())
            rg = set(range(start-1, end))
            # Check if all words in list are capital
            if rg.issubset(caps):
                yield list(range(start-1, end)), self.label

up = AllUpperNounsMatcher('UpNoun')

## Creating the candidates
To use candidates from both `Matcher` objects, we can use a `MultiMatcher`. We'll use this to extract our candidate entities from the sentences into an `Entities` object. Using both matchers together will provide very high recall, but may have poor precision. In the next demo notebook (`GeneTaggerExample_Learning.ipynb`), we'll write distant supervision rules and learn a model to improve precision.

In [6]:
M = MultiMatcher(dm, up)
E = Entities(sents, M)

We can visualize contexts for our extractions too. This may help in writing distant supervision rules in `GeneTaggerExample_Learning.ipynb`.

In [7]:
E[0].render()

<!--Provide the canvas id (twice) and the words via python string formatting here--!>
 
 
 Mutations in BCS1L , a respiratory chain complex III assembly chaperone , constitute a major cause of mitochondrial complex III deficiency and are associated with GRACILE and Bjrnstad syndromes .

Finally, we'll pickle the extracted candidates from our `Entities` object for use in `GeneTaggerExample_Learning.ipynb`.

In [8]:
E.dump_extractions('gene_tag_example/gene_tag_saved_entities.pkl')